# Comedy Bang Bang Podcast Best Of - K - Nearest Neighbors Model

In [11]:
# Attempted fix for AttributeError: 'Flags' object has no attribute 'c_contiguous'
# https://github.com/scikit-learn/scikit-learn/issues/27180#issuecomment-1694638140
# After I added the actors and characters one-hot encoded columns
# Fix doesn't work
# only needs to be run once per environment
# ! pip uninstall -y scikit-learn
# ! pip install scikit-learn==1.2.2

# ! pip uninstall -y scikit-learn
# ! pip install scikit-learn
# ! pip install --upgrade pip

In [12]:
import os
import joblib

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
# percentage of training data to use for test
TEST_SIZE = 0.33

# random seed
RANDOM_STATE = 1234

LABEL_COLUMN_NAME = "label"

In [14]:
filename = os.path.join(
    os.getcwd(), "data", "Comedy_bang_bang_podcast_dataset-v16_train.csv"
)
df = pd.read_csv(filename)
                 #, header=0)

df.head()

,actors_Adam Brody,actors_Adam Cayton-Holland,actors_Adam DeVine,actors_Adam Pally,actors_Adam Resnick,actors_Adam Scott,actors_Aimee Mann,actors_Aisling Bea,actors_Al Madrigal,actors_Alan Tudyk,...,characters_the Mailer Daemon,characters_the Pie Minister,characters_themselves,characters_Étouffée,duration_in_minutes_winsorized_normalized,hundo,month_published_int_winsorized_normalized,num_actors,num_chars,label
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0.000000,False,0.000000,3,2,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0.024390,False,0.000000,3,1,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0.012195,False,0.000000,3,0,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0.000000,False,0.000000,5,0,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0.060976,False,0.090909,3,1,False


# Verify datatypes

In [15]:
# attempted fix for AttributeError: 'Flags' object has no attribute 'c_contiguous'
# but this doesn't fix it
# for new_column_prefix in ['actors_', 'characters_']:
#     for iter_col in df.filter(like=new_column_prefix).columns:
#         df[iter_col] = df[iter_col].astype('int64')

# verify that there are only numeric and boolean datatypes left
# there should not be any strings left
for index, value in df.dtypes.items():
    assert value in [
        "float64",
        "bool",
        "int64",
    ], f"Column name {index} is not numeric or boolean- found {value}. All features at this point should be numeric or boolean. Exiting."

print("Feature datatype check passed.")
df.dtypes

Feature datatype check passed.


actors_Adam Brody                               bool
actors_Adam Cayton-Holland                      bool
actors_Adam DeVine                              bool
actors_Adam Pally                               bool
actors_Adam Resnick                             bool
                                              ...   
hundo                                           bool
month_published_int_winsorized_normalized    float64
num_actors                                     int64
num_chars                                      int64
label                                           bool
Length: 1277, dtype: object

In [16]:
k_values = [5, 10, 50, 100]



In [17]:
# get a Panda Series of the columns and number of NaNs in each one
nan_count = np.sum(df.isnull(), axis=0)

# iterate through the Series. It could be easier to just throw and exception if
# any have a value of zero.
for index, value in nan_count.items():
    assert (
        value == 0
    ), f"Column name {df.columns[index]} (index = {index}) has {value} missing values (NaN). Decision trees cannot have any missing values. Exiting."

In [18]:
y = df[LABEL_COLUMN_NAME]
X = df.drop(columns=LABEL_COLUMN_NAME, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [19]:
def train_test_knn(X_train1, X_test1, y_train1, y_test1, k):
    """
    Fit a k Nearest Neighbors classifier to the training data X_train, y_train.
    Return the accuracy of resulting predictions on the test data.
    """

    # 1. Create the KNeighborsClassifier model object below and assign to
    # variable 'model'
    model = KNeighborsClassifier(n_neighbors=k)

    # 2. Fit the model to the training data below
    model.fit(X_train1, y_train1)

    # 3. Make predictions on the test data below and assign the result to
    # the variable 'class_label_predictions'
    # produces error: AttributeError: 'NoneType' object has no attribute 'split'
    class_label_predictions = model.predict(X_test1)

    # produces error: AttributeError: 'NoneType' object has no attribute 'split'
    # class_label_predictions = model.predict(X_test1.values)

    # 4. Compute the accuracy
    acc_score = accuracy_score(y_test1, class_label_predictions)

    return acc_score, model


def train_and_save_knn_model(X_train1, y_train1, k, model_filename):
    # train_test_knn
    model = KNeighborsClassifier(n_neighbors=k)

    # Fit the model to the training data below
    model.fit(X_train1, y_train1)

    # Save it to a file
    joblib.dump(model, model_filename)

    return model


def predict_knn(model, X_predict1, k):
    """
    Fit a k Nearest Neighbors classifier to the training data X_train, y_train.
    Return the accuracy of resulting predictions on the test data.
    """

    # model = train_and_save_knn_model(X_train1, y_train1, k, 'cbb_knn.model')

    # 3. Make predictions on the test data below and assign the result to
    # the variable 'class_label_predictions'
    # https://github.com/scikit-learn/scikit-learn/issues/27180#issuecomment-1694638140
    class_label_predictions = model.predict(X_predict1)
    
    # produces error "AttributeError: 'NoneType' object has no attribute 'split'"
    # class_label_predictions = model.predict(X_predict1.values)

    return class_label_predictions

In [20]:
acc1 = []

for k1 in k_values:
    score, model = train_test_knn(X_train, X_test, y_train, y_test, k1)
    print("k=" + str(k1) + ",  score: " + str(score))
    acc1.append(float(score))

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
# Visualizing accuracy:
fig = plt.figure()
ax = fig.add_subplot(111)
p1 = sns.lineplot(x=k_values, y=acc1, color="b", marker="o", label="Full training set")

plt.title("Accuracy of the kNN predictions")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
ax.set_xlabel("k")
ax.set_ylabel("Accuracy on the test set")
plt.show()

In [ ]:
acc1_40 = []

print("Accuracy scores for full training data:")
for k_val in range(1, 41):
    score, model = train_test_knn(X_train, X_test, y_train, y_test, k_val)
    print("k=" + str(k) + ", accuracy score: " + str(score))
    acc1_40.append(float(score))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
p1 = plt.plot(list(range(1, 41)), acc1_40, "b-", label="Full training set")

plt.title("Accuracy of the kNN predictions")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
ax.set_xlabel("k")
ax.set_ylabel("Accuracy on the test set")
plt.show()

In [ ]:
# Final Predictions
final_k_value = 10

predict_filename = os.path.join(
    os.getcwd(),
    "data",
    "Comedy_bang_bang_podcast_dataset - full_dataset-v16_predict.csv",
)
df_predict = pd.read_csv(predict_filename, header=0)


X_predict = df_predict.drop(columns=LABEL_COLUMN_NAME, axis=1)

specs, model = train_test_knn(X_train, X_test, y_train, y_test, final_k_value)

predictions = predict_knn(model, X_predict, k)
print(predictions)